Uses results from aperture photometry to derive the optical depth of the atmosphere for a given night of observing.
Does the following:
- Fits star flux as a function of airmass to find an average optical depth as a function of wavelength for the night.

Still needs to be added to notebook:
- Uses optical depth with each star cube to find an F_top for each star cube
- Takes the median of F_top
- Uses F_top to find beta (photometric conversion factor)

In [ ]:
import numpy as np
from numpy import pi, r_
from astropy.io import fits
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import math
from scipy import interpolate
from scipy import optimize
from collections import OrderedDict

### Functions

In [ ]:
def F_obs(chi,tau,F_top):
    # Function to use with curve_fit() to get tau
    # F_top falls off with -tau*chi to give you F_obs
    return F_top*np.exp(-tau*chi)

def Ftop(chi,tau,F_obs):
    # returns the flux at the top of the atmosphere by correcting for the atm
    return F_obs*np.exp(tau*chi)

def flux1(mag1,mag2,flux2):
    # Useful for returning the flux of a star if it were scaled to Vega's magnitude.
    # flux1 = Star's flux as Vega
    # flux2 = Star's measured flux
    # mag1 = Vega's flux
    # mag2 = Star's flux
    return flux2*(10**(0.4*(mag2-mag1)))

def flux_relative_to_vega(mag2,flux2):
    # Same as flux1, but with Vega's magnitude hard-coded.
    # returns Star's flux as Vega
    # flux2 = Star's measured flux
    # mag2 = Star's flux
    mag1 = 0.03 # Vega's V magnitude
    return flux2*(10**(0.4*(mag2-mag1)))

In [ ]:
def tau_flux_fit(science_directory, science_list, starlist_directory, star_directory, snr_limit, wl_range):
    
    # science_directory = location of Jupiter cube
    # science_list = location+name of Jupiter image list
    # starlist_directory = location of image name lists for each cube. !!Assumes that name of star image lists are star1, star2, etc.
    # star_directory = location of star fits files
    # snr_limit = won't include fluxes below that range while calculating tau (previously used ~25)
    # tau_fit_wl_range = range of wl in nm that's considered by the program when fitting a given wl point (typical values 2-5)
    # Note: there are values for snr_limit and wl_range in Paul's thesis, but they might not be directly applicable to our observations. There's a trade-off between wavelength range and SNR in order to provide the function with enough data points to fit. Lower wl range needs a lower SNR, and vice versa. Values I used for March 2017 were snr_limit = 25, wl_range=8.
    
    # Make sure standard stars don't have significant reddening (that V mag is approximately average/representative of other magnitudes)
    # Star lists need to be named star1, star2, etc.
    # Doesn't require that you saved files of star flux from Ap_phot
    
    # Pulls information from star cubes, assuming they've already been through Ap_phot. Uses airmass, flux, and magnitude information to scale the stars to Vega, calculate an 
    
    
    wl_sci = np.linspace(470,950,241) # These are the wavelengths that tau will be derived for
    # Otherwise, do this:
    # Pull wavelengths to analyze from science cube
    '''print 'Pulling wavelength array from science images...'
    wl_sci = []
    sciencelist = np.loadtxt(science_list,dtype=str)
    for i in range(0,len(sciencelist)):
        im = fits.open(science_directory+sciencelist[i])
        if im[0].header['rfon'] == 1:
            wl_sci.append(im[0].header['lambda'])
        im.close()'''
    
    tau = []
    tau_error = []
    
    # Quieries:
    num_stars = raw_input('How many individual stars were imaged? ')
        
    num_cubes = int(raw_input('How many total cubes are there? '))
    
    V_mags = []
    for i in range(0,int(num_cubes)):
        mags = raw_input('For cube '+str(i+1)+', what is it\'s V magnitude? ')
        V_mags.append(float(mags)) # will be same length of number of star cubes
    
    airmass_plot_query = raw_input('Do you want to see a plot of airmasses for all of the star cubes? y/n ')
    snr_plot_query = raw_input('Do you want to see a plot of the SNR spectrum for each star cube? y/n ')
    flux_plot_query = raw_input('Do you want to see a plot of the stars\'s flux? y/n ')
    photometric_quality_query = raw_input('If whether an image is photometric or not is saved in the header, do you want to save that information? y/n ')
    tau_plot_query = raw_input('Do you want to see plots of optical depth fits for each wavelength? y/n ')
    
    
    
    
    # load star lists into dictionaries
    star_list_dict = OrderedDict()
    for i in range(0,num_cubes):
        star_list_dict['star'+str(i+1)+'_list'] = np.loadtxt(starlist_directory+'star'+str(i+1),dtype=str) # appends star lists in the wrong direction, but okay as long as we're consistent
        
    
    # Pull airmasses, wavelengths, star flux, and snr of each image from from each image header from each cube and assign them to a dictionary
    
    # Make empty dictionaries
    airmass_dict = OrderedDict(); airmass_wl_dict = OrderedDict(); star_snr_dict = OrderedDict(); star_flux_dict = OrderedDict()
    
    # pulling these for plotting purposes
    for i in range(0,len(star_list_dict)):
        airmass_list = []; airmass_wl_list = []; star_snr_list = []; star_flux_list = []
        for j in star_list_dict['star'+str(i+1)+'_list']: # for each star cube
            #print j # Each image in the list
            im = fits.open(star_directory+str(j))
            if im[0].header['rfon'] == 1:
                airmass_list.append(im[0].header['airmass'])
                airmass_wl_list.append(im[0].header['lambda'])
                star_snr_list.append(im[0].header['star_snr'])    
                star_flux_list.append(im[0].header['star_flx'])
            im.close()
        airmass_dict['airmass_star'+str(i+1)] = airmass_list # dictionary of airmasses
        airmass_wl_dict['airmass_wl_star'+str(i+1)] = airmass_wl_list # dictionary of wavelngths corresponding to airmasses (and other qualities appended above)
        star_snr_dict['snr_star'+str(i+1)] = star_snr_list # dictionary of SNR
        star_flux_dict['flux_star'+str(i+1)] = star_flux_list # dictionary of star flux
        
    # Plot airmass coverage
    if airmass_plot_query == 'y':
        for j in range(0,len(airmass_dict)):
            plt.plot(airmass_dict['airmass_star'+str(j+1)], airmass_wl_dict['airmass_wl_star'+str(j+1)],'.')
        plt.title('Airmass coverage')
        plt.ylabel('Wavelength')
        plt.xlabel('Airmass')
        plt.legend(airmass_dict.keys())
        plt.show()
        
    # Plot SNR spectra
    if snr_plot_query == 'y':
        for j in range(0,len(airmass_wl_dict)):
            plt.plot(airmass_wl_dict['airmass_wl_star'+str(j+1)], star_snr_dict['snr_star'+str(j+1)], '.')
        plt.title('SNR of All Cubes')
        plt.ylabel('SNR')
        plt.xlabel('Wavelength')
        plt.legend(star_snr_dict.keys())
        plt.show()
        
    # Plot flux spectra
    if flux_plot_query == 'y':
        for j in range(0,len(airmass_wl_dict)):
            plt.plot(airmass_wl_dict['airmass_wl_star'+str(j+1)], star_flux_dict['flux_star'+str(j+1)])
        plt.title('Star Flux')
        plt.ylabel('Flux (counts/sec)')
        plt.xlabel('Wavelength')
        plt.legend(star_flux_dict.keys())
        plt.show()
                
    # Scale spectra to Vega's magntiude and plot them
    for j in range(0,len(airmass_wl_dict)):
        plt.plot(airmass_wl_dict['airmass_wl_star'+str(j+1)],\
        flux_relative_to_vega(V_mags[j],np.array(star_flux_dict['flux_star'+str(j+1)])))
    plt.title('Fluxes scaled to Vega')
    plt.ylabel('DN/sec')
    plt.xlabel('Wavelength')
    plt.show()
    
    # save photometric test if done previously/desired (not a lot of evidence accounting for this improves photometric cal in the end)
    if photometric_quality_query == 'y':
        phot_qual_dict = OrderedDict()
        for i in range(0,len(star_list_dict)):
            phot_qual_list = []
            for j in star_list_dict['star'+str(i+1)+'_list']: # for each star cube
                im = fits.open(star_directory+str(j))
                if im[0].header['rfon'] == 1:
                    phot_qual_list.append(im[0].header['phottest']) # 1 if photometric, 0 if not.
                im.close()
            phot_qual_dict['phot_qual_star'+str(i+1)] = phot_qual_list # dictionary of airmasses

            
            
            
    print 'Finding an average tau value...'
    
    
    # Find tau for each science wavelength
    for i in range(0,len(wl_sci)):
        
        print wl_sci[i]
        
        flux = []
        airmass = []
        
         # loop through cubes
        for k in range(0,len(star_list_dict)):
            
            # loop through each image for a given cube
            for j in range(0,len(star_list_dict['star'+str(k+1)+'_list'])): # for each star cube
                im = fits.open(star_directory+str(star_list_dict['star'+str(k+1)+'_list'][j]))
                if im[0].header['star_snr'] >= snr_limit:
                    
                    if photometric_quality_query == 'y':
                        if im[0].header['phottest'] == 1:
                            if im[0].header['lambda'] > (wl_sci[i]-wl_range) and im[0].header['lambda'] < (wl_sci[i]+wl_range):
                                flux.append(flux_relative_to_vega(V_mags[k], im[0].header['star_flx'])) # append Vega-scaled flux
                                airmass.append(im[0].header['airmass'])
                                
                    else:
                        if im[0].header['lambda'] > (wl_sci[i]-wl_range) and im[0].header['lambda'] < (wl_sci[i]+wl_range):
                            flux.append(flux_relative_to_vega(V_mags[k], im[0].header['star_flx'])) # append Vega-scaled flux
                            airmass.append(im[0].header['airmass'])
                
                im.close()
        
        # for a given wavelength, sort airmass and flux
        dummy = np.zeros((len(airmass),2))
        dummy[:,0] = airmass
        dummy[:,1] = flux
        dummy = sorted(dummy, key=lambda a_entry: a_entry[0])
        
        #Make them lists again
        airmass_sorted = []
        flux_sorted = []
        for x in range(0,len(dummy)):
            airmass_sorted.append(dummy[x][0])
            flux_sorted.append(dummy[x][1])
        
        # Fit the values for a given wavelength using the equations defined previously
        guess = np.array((0.01,1e3)) # Does this work??
        popt, pcov = curve_fit(F_obs, airmass_sorted, flux_sorted,guess) # curve_fit(Function to fit, xdata, ydata)
        
        if tau_plot_query == 'y':
            plt.plot(airmass_sorted, F_obs(np.array(airmass_sorted),*popt), 'r-', label='fit')
            plt.plot(airmass_sorted, flux_sorted, 'b.', label='data')
            plt.title(wl_sci[i])
            plt.ylabel('flux')
            plt.xlabel('airmass')
            plt.show()
        
        number_of_points.append(float(len(airmass_sorted)))
        perr = np.sqrt(np.diag(pcov))
        #print perr[0] # error on tau

        #print 'tau: ',(popt[0]),'+-',perr[0],' ',wl_sci[i]
        tau.append(popt[0])
        tau_error.append(perr[0])
    return tau, tau_error

In [ ]:
tau_flux_fit('/Volumes/external_hd/march2017/redo/jupiter/',\
            '/Volumes/external_hd/march2017/march_2017_sciimages',\
            '/Volumes/external_hd/calstar_test/',\
            '/Volumes/external_hd/calstar_test/',25,4)